# Effect of a carbon tax for planetery boundaries

## Model

This notebook provides a numerical simulation of the effect of a potential carbon tax on variables which have been identified as 
key drivers of the processes underlying the planetary boundaries (PB) framework. The model, upon which we base our analysis, is set up along the lines of a standard economic textbook model of decentralized competitive economy where firms and consumers have preferences over outcomes and each solve maximization problems by engaging in trade with each other. The sectors (or economic actors) we have included are those that we have identified as being central drivers of the processes underlying the PB's these include e.g. the production of fossil fuel, fertilizer, agriculture, timber, water and fisheries.  To study the implications of a carbon tax within this setting we have proceeded along the following lines:
1. Characterize the problem each economic actor is trying to solve given their preferences, technology and constraints.
2. Derive the corresponding first order conditions for each actor.
3. From the first order conditions characterize the equilibrium allocation only in terms of the model variables (i.e. eliminate prices) 
4. Derive the comparitive statics of the effect of a carbon tax i.e. differentiate the equilibrium allocation fully with respect to the carbon tax.

The resulting equilibrium outcome thus indirectly reveals how each variable in the model responds to a carbon tax.

## Numerical solution method

The numerical solution is found as the solution to a system of 19 equations in 19 unknowns. This details of this equation system and how it was derived is found in the document referenced above (see equations 72-90). The solution method adopted below proceeds as follows:
1. From the equation system (72-90) all terms containing the carbon tax $\tau_E$ are moved to the left and all other terms to the right.
2. We then proceeded by rewriting the system in matrix/vector form where the rows correspond to the equations and columns are variables (e.g. the first row in the matrix holds the parameters of equation (72), the second row equation (73), etc.). The system can thus be written as $\mathbf{\tau} = \mathbf{A y}$ were $\mathbf{\tau}$ is a vector of containing the terms with the carbon tax, $\mathbf{y}$ are 19x1 vectors of outcome variables and $\mathbf{A}$ is a 19x19 matrix of parameters.
3. The solution to the system is thus found by inverting the matrix $\mathbf{A}$ and multiplying each side by this inverse i.e. $\mathbf{y} = \mathbf{A^{-1} \tau}$

## Code

The model can be simulated by highlighting each code cell below and clicking the "run button" above for each cell  
in consecutive order.

### Import the necessary computational modules

In [56]:
import numpy as np
import pandas as pd
print(pd.__version__)
import os, sys
from itertools import product
import re


0.23.4


### Construct coefficient matrix

In [24]:
def gen_coef_matrix(var_dict, biofuel_tax = 0, land_policy = False):

    v = var_dict
    vci = v['vci']
    pci = v['pci']
    #locals().update(var_dict)# Coefficient  Matrix (rows: equations, columns: variables)
    # note: python vectors starts with index zero
    coef_matrix = np.zeros((41, 41))
    #print(locals())
    # Policy variable vector (carbon tax)
    policy_vector = np.zeros((41, 1))

    ## Populate coefficient matrix and policy vector (note: document equation numbers specified in brackets)

    # Market clearing Fixed totat land: eq. [72]
    coef_matrix[0, vci['L_A']] = v['Q_LA']
    coef_matrix[0, vci['L_T']] = v['Q_LT']
    coef_matrix[0, vci['L_U']] = v['Q_LU']

    # Market clearing constraint fossil fuel: eq. [73]
    coef_matrix[1, vci['E']] = -1
    coef_matrix[1, vci['E_Eps']] = v['Q_EEps']
    coef_matrix[1, vci['E_P']] = v['Q_EP']
    coef_matrix[1, vci['E_Fi']] = v['Q_EFi']

    # Market clearing agric. output: eq. [74]
    coef_matrix[2, vci['A']] = -1
    coef_matrix[2, vci['A_B']] = v['Q_AB']
    coef_matrix[2, vci['A_F']] = v['Q_AF']

    # Market clearing energy. services: eq. [75]
    coef_matrix[3, vci['Eps']] = -1
    coef_matrix[3, vci['Eps_A']] = v['Q_EpsA']
    coef_matrix[3, vci['Eps_Y']] = v['Q_EpsY']

    # Agricultural prod. : eq. [76]
    coef_matrix[4, vci['L_A']] = v['GammaA_LA']
    coef_matrix[4, vci['A']] = -1.0
    coef_matrix[4, vci['Eps_A']] = v['GammaA_EpsA']
    coef_matrix[4, vci['P']] = v['GammaA_P']
    coef_matrix[4, vci['W']] = v['GammaA_W']
    coef_matrix[4, vci['MA']] = v['GammaA_MA']

    # Timber prod. : eq. [77]
    coef_matrix[5, vci['L_T']] = v['GammaT_LT']
    coef_matrix[5, vci['T']] = -1.0
    coef_matrix[5, vci['MT']] = v['GammaT_MT']

    # Fertilizer prod. : eq. [78]
    coef_matrix[6, vci['P']] = -1
    coef_matrix[6, vci['E_P']] = v['GammaP_EP']
    coef_matrix[6, vci['Pho']] = v['GammaP_Pho']
    coef_matrix[6, vci['MP']] = v['GammaP_MP']

    # Energy prod. : eq. [79]
    coef_matrix[7, vci['Eps']] = -1.0
    coef_matrix[7, vci['A_B']] = v['GammaEps_AB']
    coef_matrix[7, vci['E_Eps']] = v['GammaEps_EEps']
    coef_matrix[7, vci['R']] = v['GammaEps_R']

    # Final good prod. : eq. [80]
    coef_matrix[8, vci['Y']] = -1.0
    coef_matrix[8, vci['Eps_Y']] = v['GammaY_EpsY']
    coef_matrix[8, vci['MY']] = v['GammaY_MY']

    # Fisheries prod.
    coef_matrix[9, vci['Fi']] = -1.0
    coef_matrix[9, vci['E_Fi']] = v['GammaFi_EFi']
    coef_matrix[9, vci['MFi']] = v['GammaFi_MFi']

    # foc: agric. wrt land use
    coef_matrix[10, pci['p_A']] = 1
    coef_matrix[10, pci['p_L']] = -1
    coef_matrix[10, vci['L_A']] = -v['V_A'] - 1/v['sigma_A']
    coef_matrix[10, vci['A']] = 1/v['sigma_A']

    # foc: agric. wrt P (fertiliz)
    coef_matrix[11, pci['p_A']] = 1
    coef_matrix[11, pci['p_P']] = -1
    coef_matrix[11, vci['A']] = 1/v['sigma_A']
    coef_matrix[11, vci['P']] = -(1/v['sigma_A'] - 1/v['sigma_nLA'])*v['GammanLA_P'] - 1/v['sigma_nLA']
    coef_matrix[11, vci['MA']] = -(1/v['sigma_A'] - 1/v['sigma_nLA'])*v['GammanLA_MA'] 
    coef_matrix[11, vci['W']] = -(1/v['sigma_A'] - 1/v['sigma_nLA'])*v['GammanLA_W'] 
    coef_matrix[11, vci['Eps_A']] = -(1/v['sigma_A'] - 1/v['sigma_nLA'])*v['GammanLA_EpsA']

    # foc: agric. wrt P and MA
    coef_matrix[12, pci['p_P']] = 1
    coef_matrix[12, pci['p_MA']] = -1
    coef_matrix[12, vci['MA']] = -1/v['sigma_nLA']
    coef_matrix[12, vci['P']] = 1/v['sigma_nLA']

    # foc: agric. wrt P and W
    coef_matrix[13, pci['p_P']] = 1
    coef_matrix[13, pci['p_W']] = -1
    coef_matrix[13, vci['W']] = -1/v['sigma_nLA']
    coef_matrix[13, vci['P']] = 1/v['sigma_nLA']

    # foc: agric. wrt P and Eps_A
    coef_matrix[14, pci['p_P']] = 1
    coef_matrix[14, pci['p_Eps']] = -1
    coef_matrix[14, vci['Eps_A']] = -1/v['sigma_nLA']
    coef_matrix[14, vci['P']] = 1/v['sigma_nLA']

    # foc: Eps. wrt E_Eps
    coef_matrix[15, pci['p_Eps']] = 1
    coef_matrix[15, pci['p_E']] = -1
    coef_matrix[15, vci['Eps']] = 1/v['sigma_Eps']
    coef_matrix[15, vci['E_Eps']] = -1/v['sigma_Eps']

    # foc: Eps. wrt A_B
    policy_vector[16] = biofuel_tax/(1+v['tau_E'])
    coef_matrix[16, pci['p_Eps']] = 1
    coef_matrix[16, pci['p_A']] = -1
    coef_matrix[16, vci['Eps']] = 1/v['sigma_Eps']
    coef_matrix[16, vci['A_B']] = -1/v['sigma_Eps']

    # foc: Eps. wrt R
    coef_matrix[17, pci['p_Eps']] = 1
    coef_matrix[17, pci['p_R']] = -1
    coef_matrix[17, vci['Eps']] = 1/v['sigma_Eps']
    coef_matrix[17, vci['R']] = -1/v['sigma_Eps']

    # foc: P wrt E_P
    coef_matrix[18, pci['p_P']] = 1
    coef_matrix[18, pci['p_E']] = -1
    coef_matrix[18, vci['P']] = 1/v['sigma_P']
    coef_matrix[18, vci['E_P']] = -1/v['sigma_P']

    # foc: P wrt Pho
    coef_matrix[19, pci['p_P']] = 1
    coef_matrix[19, pci['p_Pho']] = -1
    coef_matrix[19, vci['P']] = 1/v['sigma_P']
    coef_matrix[19, vci['Pho']] = -1/v['sigma_P']

    # foc: P wrt MP
    coef_matrix[20, pci['p_P']] = 1
    coef_matrix[20, pci['p_MP']] = -1
    coef_matrix[20, vci['P']] = 1/v['sigma_P']
    coef_matrix[20, vci['MP']] = -1/v['sigma_P']

    # foc: T wrt L_T
    coef_matrix[21, pci['p_T']] = 1
    coef_matrix[21, pci['p_L']] = -1
    coef_matrix[21, vci['T']] = 1/v['sigma_T']
    coef_matrix[21, vci['L_T']] = -1/v['sigma_T'] - v['V_T']

    # foc: T wrt MT
    coef_matrix[22, pci['p_T']] = 1
    coef_matrix[22, pci['p_MT']] = -1
    coef_matrix[22, vci['T']] = 1/v['sigma_T']
    coef_matrix[22, vci['MT']] = -1/v['sigma_T']

    # foc: Y wrt EpsY
    coef_matrix[23, pci['p_Y']] = 1
    coef_matrix[23, pci['p_Eps']] = -1
    coef_matrix[23, vci['Y']] = 1/v['sigma_Y']
    coef_matrix[23, vci['Eps_Y']] = -1/v['sigma_Y']

    # foc: Y wrt MY
    coef_matrix[24, pci['p_Y']] = 1
    coef_matrix[24, pci['p_MY']] = -1
    coef_matrix[24, vci['Y']] = 1/v['sigma_Y']
    coef_matrix[24, vci['MY']] = -1/v['sigma_Y']

    # foc: Fi wrt E_Fi
    coef_matrix[25, pci['p_Fi']] = 1
    coef_matrix[25, pci['p_E']] = -1
    coef_matrix[25, vci['Fi']] = 1/v['sigma_Fi']
    coef_matrix[25, vci['E_Fi']] = -1/v['sigma_Fi']

    # foc: Fi wrt MFi
    coef_matrix[26, pci['p_Fi']] = 1
    coef_matrix[26, pci['p_MFi']] = -1
    coef_matrix[26, vci['Fi']] = 1/v['sigma_Fi']
    coef_matrix[26, vci['MFi']] = -1/v['sigma_Fi']

    # foc: fossil extraction
    policy_vector[27] = 1.0/(1+v['tau_E'])
    coef_matrix[27, pci['p_E']] = 1
    coef_matrix[27, vci['E']] = -v['Lambda_E'] 

    # foc: phosphate extraction
    coef_matrix[28, pci['p_Pho']] = 1
    coef_matrix[28, vci['Pho']] = -v['Lambda_Pho'] 

    # foc: water extraction
    coef_matrix[29, pci['p_W']] = 1
    coef_matrix[29, vci['W']] = -v['Lambda_W']

    # foc: renewables extraction
    coef_matrix[30, pci['p_R']] = 1
    coef_matrix[30, vci['R']] = -v['Lambda_R']

    # foc: MA extraction
    coef_matrix[31, pci['p_MA']] = 1
    coef_matrix[31, vci['MA']] = -v['Lambda_MA']

    # foc: MFi extraction
    coef_matrix[32, pci['p_MFi']] = 1
    coef_matrix[32, vci['MFi']] = -v['Lambda_MFi']

    # foc: MP extraction
    coef_matrix[33, pci['p_MP']] = 1
    coef_matrix[33, vci['MP']] = -v['Lambda_MP']

    # foc: MT extraction
    coef_matrix[34, pci['p_MT']] = 1
    coef_matrix[34, vci['MT']] = -v['Lambda_MT']

    # foc: MY extraction
    coef_matrix[35, pci['p_MY']] = 1
    coef_matrix[35, vci['MY']] = -v['Lambda_MY']

    # foc: U wrt A_F & Fi
    coef_matrix[36, pci['p_A']] = 1
    coef_matrix[36, pci['p_Fi']] = -1
    coef_matrix[36, vci['A_F']] = 1/v['sigma_F']
    coef_matrix[36, vci['Fi']] = -1/v['sigma_F']

    # foc: U wrt T & L_U
    if not land_policy:
        coef_matrix[37, pci['p_T']] = 1
        coef_matrix[37, pci['p_L']] = -1
        coef_matrix[37, vci['T']] = 1/v['sigma_nF']
        coef_matrix[37, vci['L_U']] = -1/v['sigma_nF']
    else:
        coef_matrix[37, vci['L_U']] = 1

    # foc: U wrt T & Y
    coef_matrix[38, pci['p_T']] = 1
    coef_matrix[38, pci['p_Y']] = -1
    coef_matrix[38, vci['T']] = 1/v['sigma_nF']
    coef_matrix[38, vci['Y']] = -1/v['sigma_nF']

    # foc: U wrt A_F & Y
    coef_matrix[39, pci['p_A']] = 1
    coef_matrix[39, pci['p_Y']] = -1
    coef_matrix[39, vci['A_F']] = 1/v['sigma_F'] + (1/v['sigma_U'] - 1/v['sigma_F'])*v['GammaF_AF']
    coef_matrix[39, vci['Y']] = -1/v['sigma_nF'] - (1/v['sigma_U'] - 1/v['sigma_nF'])*v['GammanF_Y']
    coef_matrix[39, vci['Fi']] = (1/v['sigma_U'] - 1/v['sigma_F'])*v['GammaF_Fi']
    coef_matrix[39, vci['T']] = -(1/v['sigma_U'] - 1/v['sigma_nF'])*v['GammanF_T']
    coef_matrix[39, vci['L_U']] = -(1/v['sigma_U'] - 1/v['sigma_nF'])*v['GammanF_LU']

    # foc: Budget
    coef_matrix[40, pci['p_A']] = v['GammaU_AF']
    coef_matrix[40, vci['A_F']] = v['GammaU_AF']
    coef_matrix[40, pci['p_Y']] = v['GammaU_Y']
    coef_matrix[40, vci['Y']] = v['GammaU_Y']
    coef_matrix[40, pci['p_Fi']] = v['GammaU_Fi']
    coef_matrix[40, vci['Fi']] = v['GammaU_Fi']
    coef_matrix[40, pci['p_L']] = v['GammaU_LU']
    coef_matrix[40, vci['L_U']] = v['GammaU_LU']
    coef_matrix[40, pci['p_T']] = v['GammaU_T']
    coef_matrix[40, vci['T']] = v['GammaU_T']

    return coef_matrix, policy_vector


### Model parameters which can be derived from the specified values above

In [25]:
def gen_subst_effects(p, mean_results, biofuel_tax):
    r = mean_results
    pci = p['pci']

    subst_effect = {k: 0 for k in p['vci'].keys()}
    sumP = p['GammaP_EP']*r[pci['p_E']] + p['GammaP_Pho']*r[pci['p_Pho']] + p['GammaP_MP']*r[pci['p_MP']]
    sumEps = p['GammaEps_EEps']*r[pci['p_E']] + p['GammaEps_AB']*(biofuel_tax/(1+p['tau_E'])+r[pci['p_A']]) + p['GammaEps_R']*r[pci['p_R']]
    
    sumY = p['GammaY_EpsY']*r[pci['p_Eps']] + p['GammaY_MY']*r[pci['p_MY']]
    sumT = p['GammaT_LT']*r[pci['p_L']] + p['GammaT_MT']*r[pci['p_MT']]
    sumFi = p['GammaFi_EFi']*r[pci['p_E']] + p['GammaFi_MFi']*r[pci['p_MFi']]
    
    sumnLA = (p['sigma_A']*p['GammaA_EpsA'] + (p['sigma_nLA']-p['sigma_A'])*p['GammanLA_EpsA'])*r[pci['p_Eps']]
    sumnLA += (p['sigma_A']*p['GammaA_W'] + (p['sigma_nLA']-p['sigma_A'])*p['GammanLA_W'])*r[pci['p_W']]
    sumnLA += (p['sigma_A']*p['GammaA_P'] + (p['sigma_nLA']-p['sigma_A'])*p['GammanLA_P'])*r[pci['p_P']]
    sumnLA += (p['sigma_A']*p['GammaA_MA'] + (p['sigma_nLA']-p['sigma_A'])*p['GammanLA_MA'])*r[pci['p_MA']]
    sumnLA += p['sigma_A']*p['GammaA_LA']*r[pci['p_L']]    

    sumLA = p['sigma_A']*(p['GammaA_EpsA']*r[pci['p_Eps']] + p['GammaA_W']*r[pci['p_W']] + p['GammaA_P']*r[pci['p_P']] + p['GammaA_MA']*r[pci['p_MA']])
    sumLA += p['sigma_A']*p['GammaA_LA']*r[pci['p_L']]

    sumF = (p['sigma_U']*p['GammaU_AF'] + (p['sigma_F']-p['sigma_U'])*p['GammaF_AF'])*r[pci['p_A']]
    sumF += (p['sigma_U']*p['GammaU_Fi'] + (p['sigma_F']-p['sigma_U'])*p['GammaF_Fi'])*r[pci['p_Fi']]
    sumF += p['sigma_U']*p['GammaU_Y']*r[pci['p_Y']] + p['sigma_U']*p['GammaU_T']*r[pci['p_T']] + p['sigma_U']*p['GammaU_LU']*r[pci['p_L']] 

    sumnF = (p['sigma_U']*p['GammaU_Y'] + (p['sigma_nF']-p['sigma_U'])*p['GammanF_Y'])*r[pci['p_Y']] 
    sumnF += (p['sigma_U']*p['GammaU_T'] + (p['sigma_nF']-p['sigma_U'])*p['GammanF_T'])*r[pci['p_T']] 
    sumnF += (p['sigma_U']*p['GammaU_LU'] + (p['sigma_nF']-p['sigma_U'])*p['GammanF_LU'])*r[pci['p_L']] 
    sumnF += p['sigma_U']*p['GammaU_AF']*r[pci['p_A']]  + p['sigma_U']*p['GammaU_Fi']*r[pci['p_Fi']] 

    subst_effect['E_P'] = p['sigma_P']*(sumP - r[pci['p_E']]) 
    subst_effect['Pho'] = p['sigma_P']*(sumP - r[pci['p_Pho']]) 
    subst_effect['MP'] = p['sigma_P']*(sumP - r[pci['p_MP']]) 
    subst_effect['E_Eps'] = p['sigma_Eps']*(sumEps - r[pci['p_E']]) 
    subst_effect['A_B'] = p['sigma_Eps']*(sumEps - (biofuel_tax/(1+p['tau_E'])+r[pci['p_A']])) 
    subst_effect['R'] = p['sigma_Eps']*(sumEps - r[pci['p_R']]) 

    subst_effect['Eps_Y'] = p['sigma_Y']*(sumY - r[pci['p_Eps']]) 
    subst_effect['MY'] = p['sigma_Y']*(sumY - r[pci['p_MY']]) 
    subst_effect['L_T'] = p['sigma_T']*(sumT - r[pci['p_L']]) 
    subst_effect['MT'] = p['sigma_T']*(sumT - r[pci['p_MT']]) 
    subst_effect['E_Fi'] = p['sigma_Fi']*(sumFi - r[pci['p_E']]) 
    subst_effect['MFi'] = p['sigma_Fi']*(sumFi - r[pci['p_MFi']]) 
                            
    subst_effect['Eps_A'] = sumnLA - p['sigma_nLA']*r[pci['p_Eps']]
    subst_effect['W'] = sumnLA - p['sigma_nLA']*r[pci['p_W']]
    subst_effect['P'] = sumnLA - p['sigma_nLA']*r[pci['p_P']]
    subst_effect['MA'] = sumnLA - p['sigma_nLA']*r[pci['p_MA']]
    subst_effect['L_A'] = sumLA - p['sigma_A']*r[pci['p_L']]

    subst_effect['A_F'] = sumF - p['sigma_F']*r[pci['p_A']]
    subst_effect['Fi'] = sumF - p['sigma_F']*r[pci['p_Fi']]

    subst_effect['Y'] = sumnF - p['sigma_nF']*r[pci['p_Y']]
    subst_effect['T'] = sumnF - p['sigma_nF']*r[pci['p_T']] 
    subst_effect['L_U'] = sumnF - p['sigma_nF']*r[pci['p_L']]
          

    return {k: (val[0] if isinstance(val, np.ndarray) else val) for k, val in subst_effect.items()}

In [54]:
var_desc_dict = {'L_A': ('$L_A$', '(land-share agriculture)'), 'L_T': ('$L_T$', '(land-share timber)'), 'L_U': ('$L_U$', '(land-share other)'),
                'E': ('$E$', '(fossil-fuel extracted)'), 'E_Eps': ('$E_{\mathcal{E}}$', '(fossil-fuel use energy serv.)'),
                'E_P':('$E_P$', '(fossil-fuel use fertilizer prod.)'), 'E_Fi': ('$E_F$', '(fossil-fuel use fisheries)'),
                'A': ('$A\;\;$', '(agriculture total production)'), 'A_B': ('$A_B$', '(agriculture prod. for biofuels)'), 
                 'A_F': ('$A_F$', '(agriculture prod. for food)'),
                'Eps': ('$\mathcal{E}\;\;$', '(energy services)'), 'Eps_A': ('$\mathcal{E}_A$', '(energy for agriculture)'), 
                'Eps_Y': ('$\mathcal{E}_{Y}$', '(energy services for final goods)'), 'P': ('$P\;\;$', '(fertilizer production)'), 'W': ('$W\;$', '(water production)'),
                'Pho': ('$\mathcal{P}\;\;$', '(phosphate extraction)'), 'R': ('$R\;\;$', '(renewables production)'), 'Fi': ('$F\;\;$','(fisheries production)'), 
                'T': ('$T\;\;$', '(timber production)'), 'Y': ('$Y\;\;$', '(final goods)'), 'MA': ('$M_A\;\;$', '(intermediaries agriculture.)'), 
                'MT': ('$M_T\;\;$', '(intermediaries timber.)'), 'MFi': ('$M_F\;\;$', '(intermediaries fisheries)'), 'MY': ('$M_Y\;\;$', '(intermediaries final goods.)'),
                'MP': ('$M_P\;\;$', '(intermediaries fertilizers)')}

# prepare list of price changes and labels
price_desc_dict =  {'p_A': ('$p_A$', '(agricultural goods)'), 'p_E': ('$p_E$', '(fossil-fuels)'), 'p_Eps': ('$p_{\mathcal{E}}$', '(energy services)'),
                    'p_Fi': ('$p_{F}$', '(fisheries)'), 'p_L': ('$p_{L}$', '(land)'), 'p_MA': ('$p_{M_A}$', '(intermediaries agric.)'), 
                    'p_MFi': ('$p_{M_F}$', '(intermediaries fisheries)'), 'p_MP': ('$p_{M_P}$', '(intermediaries fertilizers.)'), 
                    'p_MT': ('$p_{M_T}$', '(intermediaries timber)'), 'p_MY': ('$p_{M_Y}$', '(intermediaries final goods)'), 'p_P': ('$p_{P}$', '(fertilizers)'), 
                    'p_Pho': ('$p_{Pho}$', '(phosphate)'), 'p_R': ('$p_{R}$', '(renewables)'), 'p_T': ('$p_{T}$', '(timber)'), 'p_W': ('$p_{W}$', '(water)'), 
                    'p_Y': ('$p_{Y}$', '(final goods)')}

def gen_results(robust_check=True, biofuel_tax=0, land_policy=False, var_desc_dict=var_desc_dict, price_desc_dict=price_desc_dict):

    # eos -> elasticity of substitution
    sigma_U = [0.4, 0.6, 0.5] # sigma_U = 0.8     # eos: food and non food (utility) 
    sigma_F = [1.13, 1.33, 1.23]   # eos: agricult. and fish in food (nested utility)
    sigma_nF = [1.5, 2.1, 1.8] # sigma_nF = 1.35   # eos: timber, manufacturing and land products (nested utility)
    sigma_A = [1.10, 1.24, 1.14]   # eos: land and non-land (agric. prod)
    sigma_P = [0.05, 0.3, 0.2]    # sigma_P = 0.01    # eos: phosphor and fossil fuels (fertil. prod)
    sigma_nLA = [0.25, 0.75, 0.5]   # sigma_nLA = 0.05  # eos: phosphor, energy and water (nested agric. prod)
    sigma_Eps = [1.5, 2.1, 1.8]  # [1.28, 1.92, 1.6]    # eos: fossil fuel, renewables and biofuel (energy prod)
           
    sigma_Fi = [0.1, 1, 0.2]       # eos: fossil fuels, intermediate inputs
    sigma_T = [0.1, 1, 0.2]        # eos: land, intermediate inputs
    sigma_Y = [0.1, 1, 0.5]        # eos: energy services, intermediate inputs

    # supply elasticities (Lambda)
    Lambda_R = 1/2.7       # supply elast. renewables
    Lambda_E = [0.8, 1.2, 1]           # supply elast. fossil fuel
    Lambda_W = 1/1.79      # supply elast. water
    Lambda_Pho = 1/1.5     # supply elast. phosphate
    # Lambda_Fi = 1/10     # supply elast. fish

    Lambda_M = [0, 2, 1]
    Lambda_MA = Lambda_M # 1/1.7        # supply elast. intermediates to agriculture
    Lambda_MT = Lambda_M                # supply elast. intermediates to timber
    Lambda_MY = Lambda_M                # supply elast. intermediates to final goods
    Lambda_MP = Lambda_M                # supply elast. intermediates to fertilizers
    Lambda_MFi = Lambda_M               # supply elast. intermediates to fisheries


    # quantity shares (e.g. Q_LA = LA/L)
    Q_LA = 0.53         # share of land used for agriculture
    Q_LT = 0.02         # share of land used for timber
    Q_AB = 0.038        # share of agri. prod. used for biofuels prod.
    Q_EpsA = 0.05        # share of energy used for agri. prod.
    Q_EP = 0.014        # share of fossil fuel used for fertilizer prod.
    Q_EFi = 0.004       # share of fossil fuel used for fisheries prod.

    # factor shares (Gamma)
    GammaU_F = 0.1235          # factor share food (utility) x
    GammaF_Fi = 0.034          # factor share fish (food) x
    GammanF_Y = 0.991          # factor share manufacturing (non-food) x
    GammanF_LU = 0.001711      # factor share unused land (non-food) x
    GammaA_LA = 0.192          # factor share land (agric. prod) x
    GammanLA_P = 0.0796        # factor share phospor (non-land) x
    GammanLA_W = 0.0239        # factor share water (non-land) x
    GammaP_EP = 0.1095         # factor share fossil fuel (fertilizer prod.)   x
    GammaEps_AB = 0.0037       # factor share biofuels (energy services prod.)x
    GammaEps_EEps = 0.9433     # factor share fossil fuel (energy services prod.)x
    GammaFi_EFi = 0.228        # factor share fossil fuel (fisheries.) 
    GammaT_LT = 0.3748         # factor share land (timber prod.)x
    GammaY_EpsY = 0.0638       # factor share energy (manufacturing.) x
    GammanLA_EpsA = 0.0412     # factor share energy services in (non-land)  1-GammanLA_W-GammanLA_P   x
    GammaP_Pho = 0.3127        # factor share phosphate (fertilizer prod.) 1-GammaP_EP  x

    # elasticity of conversion costs : V_T = (L_T/C_T)*(dC_T/dL_T)
    V_T = 0.05
    V_A = 0.05

    # Carbon tax
    tau_E = 0.0
    # print(locals())
    
    # quantity shares (e.g. Q_LA = LA/L)
    Q_LU = 1-Q_LT-Q_LA # share of land used for recreation
    # Q_EP = 1-Q_EEps-Q_EFi    # share of fossil fuel used for fertilizer prod.

    Q_EEps = 1-Q_EFi-Q_EP    # share of fossil fuel used for fisheries prod.

    Q_AF = 1-Q_AB      # share of agri. prod. used for food prod.
    Q_EpsY = 1-Q_EpsA  # share of energy used for final goods prod.

    # factor shares (Gamma)
    GammaU_nF = 1-GammaU_F     # factor share non-food (utility)
    GammaF_AF = 1-GammaF_Fi    # factor share agricultural (food)
    GammanF_T = 1-GammanF_LU-GammanF_Y     # factor share timber (non-food)
    GammaA_nLA = 1-GammaA_LA               # factor share non-land (agric. prod)


    GammaEps_R = 1 - GammaEps_AB-GammaEps_EEps  # factor share renewables (energy services prod.)
    GammaFi_MFi = 1 - GammaFi_EFi               # factor share intermediates (energy services prod.)
    GammaT_MT = 1 - GammaT_LT                   # factor share intermediates (energy services prod.)
    GammaY_MY = 1 - GammaY_EpsY                 # factor share intermediates (energy services prod.)
    GammanLA_MA = 1 - GammanLA_W - GammanLA_P - GammanLA_EpsA    # factor share intermediates (non-land)
    GammaP_MP = 1 - GammaP_EP - GammaP_Pho                       # factor share intermediates (fertilizer prod.)

    GammaT_LTLT = GammaT_LT-1  
    GammaY_EpsYEpsY = GammaY_EpsY-1

    GammaA_P = GammaA_nLA*GammanLA_P
    GammaA_W = GammaA_nLA*GammanLA_W 
    GammaA_EpsA = GammaA_nLA*GammanLA_EpsA 
    GammaU_AF = GammaU_F*GammaF_AF
    GammaU_Fi = GammaU_F*GammaF_Fi
    GammaU_Y= GammaU_F*GammanF_Y
    GammaU_LU= GammaU_F*GammanF_LU
    GammaU_T= GammaU_F*GammanF_T
    GammaA_MA = GammaA_nLA*GammanLA_MA 

    local_variables = locals()
    local_variables.pop('robust_check', None)
    local_variables.pop('biofuel_tax', None)
    local_variables.pop('var_desc_dict', None)
    local_variables.pop('price_desc_dict', None)
    local_variables.pop('land_policy', None)
    
    #print(local_variables)

    lv_lists, lv_strings = {}, {}
    for k, val in iter(local_variables.items()):
        if isinstance(val, list):
            if robust_check:
                lv_lists[k] = val
            else: 
                # third param in list the mean
                lv_strings[k] = val[2]
        else:
            lv_strings[k] = val
    # print(lv_lists, lv_strings)
    # example: [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
    distinct_combination_sets = list(product(range(2), repeat=len(lv_lists)))
    results_matrix = np.zeros((41, len(distinct_combination_sets)))
    mean_results = np.zeros((41, 1))
    
    # print distinct_combination_sets
    # result_set = []
    # Variable order for columns of matrix an rows in the outcome vector: 
    # L_A, L_T, L_U, E, E_eps, E_P, E_Fi, A, A_B, A_F, Eps, Eps_A, Eps_Y, P, W, Pho, R, Fi, T, Y, MA, MT, MFi, MY, MP
    vci = {'L_A': 0, 'L_T': 1, 'L_U': 2, 'E': 3, 'E_Eps': 4, 'E_P': 5, 'E_Fi': 6, 'A': 7, 'A_B': 8, 'A_F': 9, 'Eps': 10, 'Eps_A': 11, 'Eps_Y': 12, 'P': 13, 'W': 14, 'Pho': 15, 'R': 16, 'Fi': 17, 'T': 18, 'Y': 19, 'MA': 20, 'MT': 21, 'MFi': 22, 'MY': 23, 'MP': 24}
    pci = {'p_A': 25, 'p_E': 26, 'p_Eps': 27, 'p_Fi': 28, 'p_L': 29, 'p_MA': 30, 'p_MFi': 31, 'p_MP': 32, 'p_MT': 33, 'p_MY': 34, 'p_P': 35, 'p_Pho': 36, 'p_R': 37, 'p_T': 38, 'p_W': 39, 'p_Y': 40}
    lv_strings['vci'] = vci
    lv_strings['pci'] = pci
    counter = 0
    for combo in distinct_combination_sets:
        combo_id = 0
        lc = {}
        for key, val in iter(lv_lists.items()):
            lc[key] = val[combo[combo_id]]
            combo_id += 1
        p = lv_strings.copy() 
        p.update(lc)
        coef_matrix, policy_vector = gen_coef_matrix(p, biofuel_tax, land_policy)
        # Invert cooeficient matrix and multiply with policy vector
        results = np.dot(np.linalg.inv(coef_matrix), policy_vector)
        results_matrix[:, counter]= results[:,0]
        counter += 1
    #print('results: ', results_matrix[:,counter-2])
    #print('counter: ', counter)
    max_results = np.amax(results_matrix, axis=1)
    min_results = np.amin(results_matrix, axis=1)
    lc = {}
    for key, val in iter(lv_lists.items()):
        lc[key] = val[2]
        combo_id += 1
    p = lv_strings.copy() 
    p.update(lc)
    
    coef_matrix, policy_vector = gen_coef_matrix(p, biofuel_tax, land_policy)
    # Invert cooeficient matrix and multiply with policy vector
    mean_results = np.dot(np.linalg.inv(coef_matrix), policy_vector)
    #print(mean_results)
    subst_effects = gen_subst_effects(p, mean_results, biofuel_tax)
    #mean_results = gen_results(robust_check = False, with_biofuel_tax = with_biofuel_tax)

    # display results in table
    sol_var_list = []
    for key, var in var_desc_dict.items():
        var_desc = var[0] + ' ' + '$\text{'+var[1]+'}$'
        sol_var_list.append({'description': var_desc, 'max value':max_results[vci[key]], 'min value':min_results[vci[key]], 'mean value':mean_results[vci[key]][0]})
    
    # display results in table
    sol_price_list = []
    for key, var in price_desc_dict.items():
        var_desc = var[0]+' '+'$\text{'+var[1]+'}$'
        sol_price_list.append({'description': var_desc, 'max value':results[pci[key]][0], 'min value':results[pci[key]][0], 'mean value':mean_results[pci[key]][0]})

    sol_substeffect_list = []
    for key, var in var_desc_dict.items():
        var_desc = var[0] + ' ' + '$\text{'+var[1]+'}$'
        price_name = 'p_' + key.split('_')[0]
        sol_substeffect_list.append({'description': var_desc, 'subst_effect': subst_effects[key], 'mean value': mean_results[vci[key]][0], 'price': mean_results[pci[price_name]][0] })

        
    df_quantities = pd.DataFrame(sol_var_list)
    df_prices = pd.DataFrame(sol_price_list)
    df_substeffect = pd.DataFrame(sol_substeffect_list)
    return df_quantities, df_prices, df_substeffect
   
df_carbontax_quantities, df_carbontax_prices, df_carbontax_substeffect = gen_results(robust_check=True, biofuel_tax=0, land_policy=False)


### Quantity changes

In [28]:
df_carbontax_quantities

,description,max value,mean value,min value
0,$L_A$ $\text{(land-share agriculture)}$,0.0526,0.0093,-0.0082
1,$L_T$ $\text{(land-share timber)}$,0.0313,0.0023,-0.0440
2,$L_U$ $\text{(land-share other)}$,0.0090,-0.0110,-0.0611
3,$E$ $\text{(fossil-fuel extracted)}$,-0.1676,-0.3573,-0.5436
4,$E_{\mathcal{E}}$ $\text{(fossil-fuel use ener...,-0.1681,-0.3610,-0.5501
5,$E_P$ $\text{(fossil-fuel use fertilizer prod.)}$,-0.0227,-0.1393,-0.2488
6,$E_F$ $\text{(fossil-fuel use fisheries)}$,-0.0919,-0.2139,-0.8997
7,$A\;\;$ $\text{(agriculture total production)}$,0.0296,-0.0057,-0.0170
8,$A_B$ $\text{(agriculture prod. for biofuels)}$,1.5165,0.6161,0.2240
9,$A_F$ $\text{(agriculture prod. for food)}$,-0.0191,-0.0303,-0.0532


### Price changes

In [29]:
df_carbontax_prices

## Solve with tax on biofuel 

In [30]:
df_biofuel_quantities, df_biofuel_prices, df_biofuel_substeffect = gen_results(robust_check=True, biofuel_tax=1)


In [31]:
df_biofuel_quantities[["description", "min value","max value","mean value" ]]

,description,min value,max value,mean value
0,$L_A$ $\text{(land-share agriculture)}$,-0.0706,-0.0018,-0.0355
1,$L_T$ $\text{(land-share timber)}$,0.0009,0.0745,0.0200
2,$L_U$ $\text{(land-share other)}$,0.0020,0.0816,0.0410
3,$E$ $\text{(fossil-fuel extracted)}$,-0.5542,-0.1715,-0.3623
4,$E_{\mathcal{E}}$ $\text{(fossil-fuel use ener...,-0.5599,-0.1721,-0.3655
5,$E_P$ $\text{(fossil-fuel use fertilizer prod.)}$,-0.3050,-0.0624,-0.1815
6,$E_F$ $\text{(fossil-fuel use fisheries)}$,-0.9148,-0.0916,-0.2215
7,$A\;\;$ $\text{(agriculture total production)}$,-0.0809,-0.0235,-0.0432
8,$A_B$ $\text{(agriculture prod. for biofuels)}$,-1.6356,-0.4152,-0.9466
9,$A_F$ $\text{(agriculture prod. for food)}$,-0.0383,0.0175,-0.0076


### Quantity changes with biofuel policy

In [32]:
df_biofuel_quantities

,description,max value,mean value,min value
0,$L_A$ $\text{(land-share agriculture)}$,-0.0018,-0.0355,-0.0706
1,$L_T$ $\text{(land-share timber)}$,0.0745,0.0200,0.0009
2,$L_U$ $\text{(land-share other)}$,0.0816,0.0410,0.0020
3,$E$ $\text{(fossil-fuel extracted)}$,-0.1715,-0.3623,-0.5542
4,$E_{\mathcal{E}}$ $\text{(fossil-fuel use ener...,-0.1721,-0.3655,-0.5599
5,$E_P$ $\text{(fossil-fuel use fertilizer prod.)}$,-0.0624,-0.1815,-0.3050
6,$E_F$ $\text{(fossil-fuel use fisheries)}$,-0.0916,-0.2215,-0.9148
7,$A\;\;$ $\text{(agriculture total production)}$,-0.0235,-0.0432,-0.0809
8,$A_B$ $\text{(agriculture prod. for biofuels)}$,-0.4152,-0.9466,-1.6356
9,$A_F$ $\text{(agriculture prod. for food)}$,0.0175,-0.0076,-0.0383


### Price changes with biofuel policy

In [33]:
df_biofuel_prices #.to_excel('carbon+biofuel_prices.xlsx')

,description,max value,mean value,min value
0,$p_A$ $\text{(agricultural goods)}$,-0.0225,0.0009,-0.0225
1,$p_E$ $\text{(fossil-fuels)}$,0.4583,0.6377,0.4583
2,$p_{\mathcal{E}}$ $\text{(energy services)}$,0.4415,0.6133,0.4415
3,$p_{F}$ $\text{(fisheries)}$,0.0819,0.0850,0.0819
4,$p_{L}$ $\text{(land)}$,-0.0091,-0.0041,-0.0091
5,$p_{M_A}$ $\text{(intermediaries agric.)}$,-0.0495,-0.0292,-0.0495
6,$p_{M_F}$ $\text{(intermediaries fisheries)}$,-0.0293,-0.0783,-0.0293
7,$p_{M_P}$ $\text{(intermediaries fertilizers.)}$,-0.0752,-0.0450,-0.0752
8,$p_{M_T}$ $\text{(intermediaries timber)}$,0.0235,0.0161,0.0235
9,$p_{M_Y}$ $\text{(intermediaries final goods)}$,0.0069,-0.0063,0.0069


In [34]:
df_biofuel_substeffect

,description,mean value,price,subst_effect
0,$L_A$ $\text{(land-share agriculture)}$,-0.0355,-0.0041,0.0061
1,$L_T$ $\text{(land-share timber)}$,0.0200,-0.0041,0.0025
2,$L_U$ $\text{(land-share other)}$,0.0410,-0.0041,0.0524
3,$E$ $\text{(fossil-fuel extracted)}$,-0.3623,0.6377,0.0000
4,$E_{\mathcal{E}}$ $\text{(fossil-fuel use ener...,-0.3655,0.6377,-0.0390
5,$E_P$ $\text{(fossil-fuel use fertilizer prod.)}$,-0.1815,0.6377,-0.1208
6,$E_F$ $\text{(fossil-fuel use fisheries)}$,-0.2215,0.6377,-0.1105
7,$A\;\;$ $\text{(agriculture total production)}$,-0.0432,0.0009,0.0000
8,$A_B$ $\text{(agriculture prod. for biofuels)}$,-0.9466,0.0009,-0.6202
9,$A_F$ $\text{(agriculture prod. for food)}$,-0.0076,0.0009,0.0039


## Experimental: Solve holding $L_U$ fixed

In [35]:
#df_land_quantities, df_land_prices, df_land_substeffect = gen_results(robust_check=True, biofuel_tax=0, land_policy=True)


### Price changes

In [36]:
#df_land_prices

### Quantity changes

In [37]:
#df_land_quantities

# For PNAS Latex Table and Figure

In [38]:

df_full = df_carbontax_quantities.copy().rename(columns={'mean value': 'outcome'}).loc[:, ['description', 'outcome']]
df_price_full = df_carbontax_prices.rename(columns={'mean value': 'prices'}).loc[:, ['description', 'prices']]

df_full['outcome_biofuel'] = df_biofuel_quantities.reset_index().rename(columns={'mean value': 'outcome_biofuel'})['outcome_biofuel']
df_price_full['prices_biofuel'] = df_biofuel_prices['mean value']

#df_full['outcome_land'] = df_land_quantities.reset_index().rename(columns={'mean value': 'outcome_land'})['outcome_land']
#df_price_full['prices_land'] = df_land_prices['mean value']

df_full = df_full.rename(columns={'description': 'index'})
df_price_full = df_price_full.rename(columns={'description': 'index'})

df_full

,index,outcome,outcome_biofuel
0,$L_A$ $\text{(land-share agriculture)}$,0.0093,-0.0355
1,$L_T$ $\text{(land-share timber)}$,0.0023,0.0200
2,$L_U$ $\text{(land-share other)}$,-0.0110,0.0410
3,$E$ $\text{(fossil-fuel extracted)}$,-0.3573,-0.3623
4,$E_{\mathcal{E}}$ $\text{(fossil-fuel use ener...,-0.3610,-0.3655
5,$E_P$ $\text{(fossil-fuel use fertilizer prod.)}$,-0.1393,-0.1815
6,$E_F$ $\text{(fossil-fuel use fisheries)}$,-0.2139,-0.2215
7,$A\;\;$ $\text{(agriculture total production)}$,-0.0057,-0.0432
8,$A_B$ $\text{(agriculture prod. for biofuels)}$,0.6161,-0.9466
9,$A_F$ $\text{(agriculture prod. for food)}$,-0.0303,-0.0076


### Generate model variables result table

In [39]:
import importlib
from web_model import pb_table_generators as pt
importlib.reload(pt)
t = pt.model_variable_result_table(var_desc_dict, price_desc_dict, df_full, df_price_full)

\begin{tabular}{lrrrr}
\hline
& \multicolumn{2}{c}{}  & \multicolumn{2}{c}{\textit{Carbon Tax +}}  \\ 
& \multicolumn{2}{c}{\textit{Carbon tax}}  & \multicolumn{2}{c}{\textit{Biofuel policy} }  \\ 
\textit{Variable} & \textit{Quantity} & \textit{Price} & \textit{Quantity} & \textit{Price}  \\ 
\hline
\multicolumn{5}{l}{\textit{Agricultural Sector: Production}} \\ 
\hspace{0.2cm}Total & -0.006 & 0.032 & -0.043 & 0.001 \\ 
\hspace{0.2cm}Biofuels & 0.616 & 0.032 & -0.947 & 0.001 \\ 
\hspace{0.2cm}Food & -0.03 & 0.032 & -0.008 & 0.001 \\ 
\multicolumn{5}{l}{\textit{Agricultural Sector: Inputs}} \\ 
\hspace{0.2cm}Land-Share Agriculture & 0.009 & 0.018 & -0.036 & -0.004 \\ 
\hspace{0.2cm}Energy in Agriculture & -0.299 & 0.615 & -0.35 & 0.613 \\ 
\hspace{0.2cm}Fertilizer Production & -0.023 & 0.063 & -0.061 & 0.034 \\ 
\hspace{0.2cm}Water Production & 0.006 & 0.004 & -0.034 & -0.019 \\ 
\multicolumn{5}{l}{\textit{Energy-related sectors and services}} \\ 
\hspace{0.2cm}Fossil-Fuel in Energy Se

# PB impacts

In [40]:
# Set policy outcome dataframe from which to generate an impact analysis

# Use: df_carbontax_quantities
df_base_policy = df_carbontax_quantities.rename(columns={'mean value': 'outcome', 'description': 'index'})[['index', 'outcome']].apply(lambda x: pd.Series([re.search(r'\((.*?)\)', x['index']).group(1).title(), re.findall("\$(.*?)\$", x['index'])[0].replace('\;', ''), x['outcome']], index=['index', 'variable', 'outcome']), axis=1)

# Use: df_biofuel_quantities
#df_base_policy = df_biofuel_quantities.rename(columns={'mean value': 'outcome', 'description': 'index'})[['index', 'outcome']].apply(lambda x: pd.Series([re.search(r'\((.*?)\)', x['index']).group(1).title(), re.findall("\$(.*?)\$", x['index'])[0].replace('\;', ''), x['outcome']], index=['index', 'variable', 'outcome']), axis=1)

# Use: df_land_quantities
#df_base_policy = df_land_quantities.rename(columns={'mean value': 'outcome', 'description': 'index'})[['index', 'outcome']].apply(lambda x: pd.Series([re.search(r'\((.*?)\)', x['index']).group(1).title(), x['outcome']], index=['index', 'outcome']), axis=1)

df_base_policy

,index,variable,outcome
0,Land-Share Agriculture,L_A,0.0093
1,Land-Share Timber,L_T,0.0023
2,Land-Share Other,L_U,-0.0110
3,Fossil-Fuel Extracted,E,-0.3573
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393
6,Fossil-Fuel Use Fisheries,E_F,-0.2139
7,Agriculture Total Production,A,-0.0057
8,Agriculture Prod. For Biofuels,A_B,0.6161
9,Agriculture Prod. For Food,A_F,-0.0303


## Aersols

The impact of aerosols in terms of their effect on climate change can be estimated via radiative forcing (see table 8.4 in
https://www.ipcc.ch/site/assets/uploads/2018/02/WG1AR5_Chapter08_FINAL.pdf). As to the PB impact we adopt the aersol optical
depth measure used in Steffen et al. we base our estinates on data from 3 sources:
1) Streets et al:  Anthropogenic and natural contributions to regional trends inaerosol optical depth, 1980–2006: https://pdfs.semanticscholar.org/8dbf/77f8c55ce39c7654a78ebe129098054fc47b.pdf
2) Lamarque. Historical (1850–2000) gridded anthropogenic and biomass burningemissions http://pure.iiasa.ac.at/id/eprint/9279/1/acp-10-7017-2010.pdf
3) Levine. Biomass Burning: The Cycling of Gases and Particulates from the Biosphere to the Atmosphere https://www-sciencedirect-com.ezp.sub.su.se/science/article/pii/B0080437516041438

Spurious Notes: 

In interpretation of results note that radiative forcing and AOD (control variable in PB framework) are inversely related (see e.g. /http://www.aaqr.org/files/article/6789/5_AAQR-17-12-AC3-0600_38-48.pdf ; https://www.nist.gov/sites/default/files/documents/mml/opt_prop_schwartz.pdf ; http://cdn.intechopen.com/pdfs/38765/InTech-Aerosol_direct_radiative_forcing_a_review.pdf)

 From https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/pdfs/energy-economics/statistical-review/bp-stats-review-2019-full-report.pdf we have that: Biofuels: 95371 Thousand tonnes oil equivalent Fossilfuels (Oil, gas coal): 4662.1 + 3309.4 + 3772.1 Million tonnes oil equivalent.


### From Streets et al we can extract global AOD average measures.

In [41]:
Streets_table2 = pd.read_csv('Streets_table2.csv', delimiter=';', index_col='Region')    # Emissions, Total Mass Burden, and AOD by Region for 2001
Streets_table4 = pd.read_csv('Streets_table4.csv', delimiter=';', index_col='Region')    # Average Contributions of Aerosol Types to Estimated AOD From 1980 to 2006
Streets_table2 = Streets_table2.loc[Streets_table2.Parameter=='AOD']
#df_aerosols = pd.read_csv('Streets_table2.txt', sep='\t', lineterminator='\r')
df_aerosols = Streets_table2.copy()
for idx, row in df_aerosols.iterrows():
    df_aerosols.at[idx, 'Sulfur'] = row['Sulfur']*Streets_table4.loc[idx, 'Sulfur-Anthro']/100
    df_aerosols.at[idx, 'BC'] = row['BC']*Streets_table4.loc[idx, 'BC-Anthro']/100
    df_aerosols.at[idx, 'OC'] = row['OC']*Streets_table4.loc[idx, 'OC-Anthro']/100

In [42]:
print(df_aerosols[["Sulfur", "BC", "OC"]].mean(axis=0))
df_aerosols


Sulfur                 0.0392
BC                     0.0003
OC                     0.0011
dtype: float64


,Parameter,Sulfur,BC,OC,Sea Salt,Dust
Region,,,,,,
United States,AOD,0.0466,0.0000,0.0003,0.0000,0.0300
South America,AOD,0.0092,0.0000,0.0007,0.0100,0.0100
OECD Europe,AOD,0.0509,0.0003,0.0003,0.0200,0.0500
Russia,AOD,0.1037,0.0003,0.0004,0.0100,0.0600
Southern Africa,AOD,0.0106,0.0003,0.0016,0.0100,0.0100
South Asia,AOD,0.0285,0.0006,0.0039,0.0100,0.0600
East Asia,AOD,0.0504,0.0004,0.0011,0.0100,0.0800
Southeast Asia,AOD,0.0139,0.0003,0.0008,0.0100,0.0100


### From: Lamarque et al.: 1850–2000 gridded anthropogenic and biomass burning emissions

In [43]:
Biomass_burning_emissions = {'BC': 2.61, 'OC': 23.25, 'SO2': 3.84 } # From table 9. in J.-F. Lamarque et al.: 1850–2000 gridded anthropogenic and biomass burning emissions
Global_anthropogenic_emissions = {'BC': 5.02, 'OC': 12.56, 'SO2': 92.71 } # From table 7 in  J.-F. Lamarque et al.: 1850–2000 gridded anthropogenic and biomass burning emissions

Global_anthropogenic_biomass_burning_emissions = {k: v*0.9 for k, v in Biomass_burning_emissions.items()}
# Note: 0.9 is a measure of percentage of biomass burning from land-use change in https://www-sciencedirect-com.ezp.sub.su.se/science/article/pii/B0080437516041438

Global_anthropogenic_incl_biomass_burning_emissions = {k: v+Global_anthropogenic_biomass_burning_emissions[k] for k, v in Global_anthropogenic_emissions.items()}

Share_Global_anthropogenic_biomass_burning_emissions = {k: Global_anthropogenic_biomass_burning_emissions[k]/v for k, v in Global_anthropogenic_incl_biomass_burning_emissions.items()}
sg = Share_Global_anthropogenic_biomass_burning_emissions
sum(list(sg.values()))

0.9796123431858057

### Compute global anthropogenic AOD 

In [44]:
df_aerosols_mean = df_aerosols[["Sulfur", "BC", "OC"]].mean(axis=0)
df_aerosols_mean_from_fossil_and_bio_fuels = df_aerosols_mean*np.array([1-sg['SO2'], 1-sg['BC'], 1-sg['OC']])
df_aerosols_mean_from_biomass = df_aerosols_mean - df_aerosols_mean_from_fossil_and_bio_fuels

AOD_global_fbf = df_aerosols_mean_from_fossil_and_bio_fuels.sum()
AOD_global_biomass = df_aerosols_mean_from_biomass.sum()
print(AOD_global_fbf)
print(AOD_global_biomass)

0.038414748418532686
0.0022015015814673216


In [45]:
Share_of_AOD_attributed_to_biofuels = AOD_global_biomass/(AOD_global_biomass+AOD_global_fbf)
Share_of_AOD_attributed_to_biofuels

0.05420248254005038

### Compute Aerosol effect from policy

In [46]:
Q_EP = 0.014        # share of fossil fuel used for fertilizer prod.
Q_EFi = 0.004       # share of fossil fuel used for fisheries prod.
Q_EEps = 1 - Q_EFi -Q_EP
Q_bio = 95371/(1000*(4662.1 + 3309.4 + 3772.1)+95371)   # Share of biofuels in tot biofuel+fossil prod.
Q_ff = 1-Q_bio
 
# 'Aerosol effect': the net change in radiative forcing from imposing a carbon tax (note: a positive value implies a increase in radiative forcing aka a decrease of AOD)
df_base_policy.loc[:, 'Aerosol effect'] = 0
df_base_policy.loc[2, 'Aerosol effect'] = -AOD_global_biomass*df_base_policy.loc[2, 'outcome']          # Natural land 
df_base_policy.loc[4, 'Aerosol effect'] = AOD_global_fbf*df_base_policy.loc[4, 'outcome']*Q_ff*Q_EEps  # Fossil fuel consump. energy services
df_base_policy.loc[5, 'Aerosol effect'] = AOD_global_fbf*df_base_policy.loc[5, 'outcome']*Q_ff*Q_EP    # Fossil fuel consump.  Fertilizer Prod
df_base_policy.loc[6, 'Aerosol effect'] = AOD_global_fbf*df_base_policy.loc[6, 'outcome']*Q_ff*Q_EFi  # Fossil fuel consump. fisheries   
df_base_policy.loc[8, 'Aerosol effect'] = AOD_global_fbf*df_base_policy.loc[8, 'outcome']*Q_bio    # Biofuel fuel prod

print('Total effect:', round(df_base_policy['Aerosol effect'].sum(), 5), '%')
print('Total effect (AOD):', (AOD_global_fbf+AOD_global_biomass)*df_base_policy['Aerosol effect'].sum()/100)
df_base_policy



Total effect: -0.0134 %
Total effect (AOD): -5.442369280271906e-06


,index,variable,outcome,Aerosol effect
0,Land-Share Agriculture,L_A,0.0093,0.0000
1,Land-Share Timber,L_T,0.0023,0.0000
2,Land-Share Other,L_U,-0.0110,0.0000
3,Fossil-Fuel Extracted,E,-0.3573,0.0000
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610,-0.0135
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393,-0.0001
6,Fossil-Fuel Use Fisheries,E_F,-0.2139,-0.0000
7,Agriculture Total Production,A,-0.0057,0.0000
8,Agriculture Prod. For Biofuels,A_B,0.6161,0.0002
9,Agriculture Prod. For Food,A_F,-0.0303,0.0000


## Climate impacts
### Methodology

In order to calculate the climate impact from the change in model variables resulting from the carbon tax we use data from two sources http://www.wri.org/sites/default/files/world_ghg_flow_chart_2005.png and https://www.nature.com/news/one-third-of-our-greenhouse-gas-emissions-come-from-agriculture-1.11708. From the graph of sectorial CO2 emissions from WRI we make the following assumptions in terms of how this connects to our model varaibles. 

* From the energy related emissions in the WRI graph (a total 66.5% = 29,36 GtCO2 eq). We wish to split this total into emissions from energy service production, fossil fuel extraction and emissions from fertilizer production. From the WRI graph we know that 6.4%+1.3%+2.2%=9.9% (3.98 GtCO2 eq) the total energy related emissions is due to extraction processes. From the Nature article we have that 0.575 GtCO2 eq is estimated to be released from fertilizer production. Hence we split the total energy related emission of 29,36 GtCO2 eq as follows
  * 24,41 GtCO2 eq are connected to the energy services variable in our model. (55.3%)
  * 3.98 GtCO2 eq are connected to the fossil fuel extraction variable in our model. (9.9%)
  * 0.575 GtCO2 eq are connected to fertilizer production. (1.3%)
* Emissions from industrial processes are linked to Final goods production in our model (In total 4.3% = 1.9 GtCO2 eq)
* Emission from land-use change are linked to the change in natural land in our model (12.2% = 5.387 GtCO2 eq).
* Emissions from agriculture are linked to total agricultural production in our model (13.8% = 6.093 GtCO2 eq)
* Emissions from fisheries in 2005 where approximately 0.14 GtCO2 (0.32%) (see https://s3-us-west-2.amazonaws.com/legacy.seaaroundus/researcher/dpauly/PDF/2019/Journal+Articles/Greer+et+al%2C+2019%2C+CO2.pdf)

Totals 97.4 % of total emissions.

In [47]:
# 44.153 GtCO2 in 2005  
# https://www.ipcc.ch/pdf/assessment-report/ar5/wg3/ipcc_wg3_ar5_technical-summary.pdf
total_emissions=44.153
df_base_policy.loc[:, 'CO2 effect'] = 0
df_base_policy.loc[2, 'CO2 effect'] = -1*5.387*df_base_policy.loc[2, 'outcome']/total_emissions  # change in natural land 
df_base_policy.loc[3, 'CO2 effect'] = 3.98*df_base_policy.loc[3, 'outcome']/total_emissions     # fossil fuel extraction 
df_base_policy.loc[4, 'CO2 effect'] = 25.960*df_base_policy.loc[4, 'outcome']/total_emissions    # Fossil-Fuel Use Energy Serv.
df_base_policy.loc[5, 'CO2 effect'] = 0.575*df_base_policy.loc[5, 'outcome']/total_emissions    # Fossils in fertilizer production
df_base_policy.loc[6, 'CO2 effect'] = 0.14*df_base_policy.loc[6, 'outcome']/total_emissions    # Fossils in fisheries
df_base_policy.loc[7, 'CO2 effect'] = 6.093*df_base_policy.loc[7, 'outcome']/total_emissions     # Emissions from Agriculture
df_base_policy.loc[19, 'CO2 effect'] = 1.9*df_base_policy.loc[19, 'outcome']/total_emissions   # Fossil-Fuel Use Final goods
# df_base_policy.loc[:, 'CO2 effect'] = df_base_policy.loc[:, 'CO2 effect']/100
total_co2_effect = round(sum(df_base_policy['CO2 effect']), 4)

print('Total CO2 effect percent: ', total_co2_effect, ' % of total')
print('Total CO2 GtCO2: ', total_co2_effect*total_emissions/100, ' GtCO2')

df_base_policy

Total CO2 effect percent:  -0.2477  % of total
Total CO2 GtCO2:  -0.10936698099999999  GtCO2


,index,variable,outcome,Aerosol effect,CO2 effect
0,Land-Share Agriculture,L_A,0.0093,0.0000,0.0000
1,Land-Share Timber,L_T,0.0023,0.0000,0.0000
2,Land-Share Other,L_U,-0.0110,0.0000,0.0013
3,Fossil-Fuel Extracted,E,-0.3573,0.0000,-0.0322
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610,-0.0135,-0.2122
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393,-0.0001,-0.0018
6,Fossil-Fuel Use Fisheries,E_F,-0.2139,-0.0000,-0.0007
7,Agriculture Total Production,A,-0.0057,0.0000,-0.0008
8,Agriculture Prod. For Biofuels,A_B,0.6161,0.0002,0.0000
9,Agriculture Prod. For Food,A_F,-0.0303,0.0000,0.0000


## Biodiversity impacts

For biodiversity impacts we use the results from the Nature article https://www.nature.com/news/biodiversity-the-ravages-of-guns-nets-and-bulldozers-1.20381. 

Here we have connected the categories of threatened species by driver in the article to our model variables and multiplied the numbers in the article by the percentage change in our model variables.

30/10-2019: We skip biodiversity effect of climate since we wish to avoid unit dependencies. For example, the strength biodiversity effect of climate would be contingent on the time period we are considering (due to larger emissions for a larger time span).

In [48]:
total_num_threats = 25779   # Non-mutually exclusive sum 

df_base_policy.loc[:, 'biodiv-val'] = 0
df_base_policy.loc[3, 'biodiv-val'] = 56    # Energy production (OIL and GAS)
df_base_policy.loc[16, 'biodiv-val'] = 56   # Energy production (Renewable Energy)
df_base_policy.loc[17, 'biodiv-val'] = 1118 # Over exploitation (Fishing)
df_base_policy.loc[18, 'biodiv-val'] = 4049 # Over exploitation (Logging)
df_base_policy.loc[7, 'biodiv-val'] = 5407-112  # Agricultural activity
df_base_policy.loc[13, 'biodiv-val'] = 1523     # Pollution (Agriculture)
df_base_policy.loc[19, 'biodiv-val'] = 907+(1901-1523)+236+1219+833      # Urban dev. (industrial) + Pollution (except Agriculture) + Human dist. (work) + Transport + Energy production (Mining)
df_base_policy.loc[:, 'Biodiv. effect'] = df_base_policy['outcome']*df_base_policy['biodiv-val']/total_num_threats
df_base_policy.loc[:, 'Biodiv. climate effect'] = 0*df_base_policy['CO2 effect']*1688/total_num_threats
df_base_policy.loc[:, 'Biodiv. incl. climate effect'] = 100*((1+df_base_policy.loc[:, 'Biodiv. effect']/100)*(1+df_base_policy.loc[:, 'Biodiv. climate effect']/100)-1)

# total_effect_biodiversity = round(sum(df_base_policy['Biodiv. effect']), 4)
# print('Total effect on biodiversity: ', total_effect_biodiversity, 'species')

# total_effect_biodiversity_incl_climate = total_effect_biodiversity + 1688*total_co2_effect_percent
# print('Total effect on biodiversity incl. climate change: ', total_effect_biodiversity_incl_climate, 'species')

# print('Total effect on biodiversity incl. climate change percent: ', total_effect_biodiversity_incl_climate/total_num_threatend_species, '%')
df_base_policy = df_base_policy.drop(columns=['biodiv-val'])

print('Total effect:', round(df_base_policy['Biodiv. incl. climate effect'].sum(), 5), '%')

df_base_policy


Total effect: -0.01054 %


,index,variable,outcome,Aerosol effect,CO2 effect,Biodiv. effect,Biodiv. climate effect,Biodiv. incl. climate effect
0,Land-Share Agriculture,L_A,0.0093,0.0000,0.0000,0.0000,0.0000,0.0000
1,Land-Share Timber,L_T,0.0023,0.0000,0.0000,0.0000,0.0000,0.0000
2,Land-Share Other,L_U,-0.0110,0.0000,0.0013,-0.0000,0.0000,0.0000
3,Fossil-Fuel Extracted,E,-0.3573,0.0000,-0.0322,-0.0008,-0.0000,-0.0008
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610,-0.0135,-0.2122,-0.0000,-0.0000,0.0000
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393,-0.0001,-0.0018,-0.0000,-0.0000,0.0000
6,Fossil-Fuel Use Fisheries,E_F,-0.2139,-0.0000,-0.0007,-0.0000,-0.0000,0.0000
7,Agriculture Total Production,A,-0.0057,0.0000,-0.0008,-0.0012,-0.0000,-0.0012
8,Agriculture Prod. For Biofuels,A_B,0.6161,0.0002,0.0000,0.0000,0.0000,0.0000
9,Agriculture Prod. For Food,A_F,-0.0303,0.0000,0.0000,-0.0000,0.0000,0.0000


## Biogeochemical 

We assume nitrogen flows are proportional to fossil fuel use. Hence a percentage increase in fossil fuel use in fertilizer production is translated to the same percentage increase in the nitrogen flow. Concerning phosphate this related one-to-one with phosphate extraction in our model. The results are: 

In [49]:
df_base_policy.loc[:, 'Biogeochem. effect'] = 0
df_base_policy.loc[5, 'Biogeochem. effect'] = df_base_policy.loc[5, 'outcome']
df_base_policy.loc[15, 'Biogeochem. effect'] = df_base_policy.loc[15, 'outcome']

print('Total Phosp. effect percent: ', round(df_base_policy.loc[15, 'Biogeochem. effect'], 4), '%')
print('Total Nitrog. effect percent: ', round(df_base_policy.loc[5, 'Biogeochem. effect'], 4), ' %')

print('Total Phosp. effect (Gg P y-1): ', round(14000*df_base_policy.loc[15, 'Biogeochem. effect']/100, 4), 'Gg P y-1')
print('Total Nitrog. effect (Tg P y-1): ', round(150*df_base_policy.loc[5, 'Biogeochem. effect']/100, 4), 'Tg P y-1')

df_base_policy

Total Phosp. effect percent:  -0.0095 %
Total Nitrog. effect percent:  -0.1393  %
Total Phosp. effect (Gg P y-1):  -1.3236 Gg P y-1
Total Nitrog. effect (Tg P y-1):  -0.2089 Tg P y-1


,index,variable,outcome,Aerosol effect,CO2 effect,Biodiv. effect,Biodiv. climate effect,Biodiv. incl. climate effect,Biogeochem. effect
0,Land-Share Agriculture,L_A,0.0093,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,Land-Share Timber,L_T,0.0023,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,Land-Share Other,L_U,-0.0110,0.0000,0.0013,-0.0000,0.0000,0.0000,0.0000
3,Fossil-Fuel Extracted,E,-0.3573,0.0000,-0.0322,-0.0008,-0.0000,-0.0008,0.0000
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610,-0.0135,-0.2122,-0.0000,-0.0000,0.0000,0.0000
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393,-0.0001,-0.0018,-0.0000,-0.0000,0.0000,-0.1393
6,Fossil-Fuel Use Fisheries,E_F,-0.2139,-0.0000,-0.0007,-0.0000,-0.0000,0.0000,0.0000
7,Agriculture Total Production,A,-0.0057,0.0000,-0.0008,-0.0012,-0.0000,-0.0012,0.0000
8,Agriculture Prod. For Biofuels,A_B,0.6161,0.0002,0.0000,0.0000,0.0000,0.0000,0.0000
9,Agriculture Prod. For Food,A_F,-0.0303,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000


## Water

In [50]:
df_base_policy.loc[:, 'Freshwater effect'] = 0
df_base_policy.loc[14, 'Freshwater effect'] = df_base_policy.loc[14, 'outcome']

print('Total Water effect percent: ', round(df_base_policy.loc[14, 'Freshwater effect'], 4), '%')
print('Total Water. effect (km3 yr-1): ', round(2600*df_base_policy.loc[14, 'Freshwater effect']/100, 4), 'km3 yr-1')

df_base_policy

Total Water effect percent:  0.0065 %
Total Water. effect (km3 yr-1):  0.1678 km3 yr-1


,index,variable,outcome,Aerosol effect,CO2 effect,Biodiv. effect,Biodiv. climate effect,Biodiv. incl. climate effect,Biogeochem. effect,Freshwater effect
0,Land-Share Agriculture,L_A,0.0093,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,Land-Share Timber,L_T,0.0023,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,Land-Share Other,L_U,-0.0110,0.0000,0.0013,-0.0000,0.0000,0.0000,0.0000,0.0000
3,Fossil-Fuel Extracted,E,-0.3573,0.0000,-0.0322,-0.0008,-0.0000,-0.0008,0.0000,0.0000
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610,-0.0135,-0.2122,-0.0000,-0.0000,0.0000,0.0000,0.0000
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393,-0.0001,-0.0018,-0.0000,-0.0000,0.0000,-0.1393,0.0000
6,Fossil-Fuel Use Fisheries,E_F,-0.2139,-0.0000,-0.0007,-0.0000,-0.0000,0.0000,0.0000,0.0000
7,Agriculture Total Production,A,-0.0057,0.0000,-0.0008,-0.0012,-0.0000,-0.0012,0.0000,0.0000
8,Agriculture Prod. For Biofuels,A_B,0.6161,0.0002,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,Agriculture Prod. For Food,A_F,-0.0303,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000


## Ocean acidification
We estimate that roughly 30% of CO2 disolves in oceans.

In [51]:
df_base_policy.loc[:, 'Ocean acid. effect'] = 0
df_base_policy.loc[:, 'Ocean acid. effect'] = df_base_policy.loc[:, 'CO2 effect']
df_base_policy

,index,variable,outcome,Aerosol effect,CO2 effect,Biodiv. effect,Biodiv. climate effect,Biodiv. incl. climate effect,Biogeochem. effect,Freshwater effect,Ocean acid. effect
0,Land-Share Agriculture,L_A,0.0093,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,Land-Share Timber,L_T,0.0023,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,Land-Share Other,L_U,-0.0110,0.0000,0.0013,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0013
3,Fossil-Fuel Extracted,E,-0.3573,0.0000,-0.0322,-0.0008,-0.0000,-0.0008,0.0000,0.0000,-0.0322
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610,-0.0135,-0.2122,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.2122
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393,-0.0001,-0.0018,-0.0000,-0.0000,0.0000,-0.1393,0.0000,-0.0018
6,Fossil-Fuel Use Fisheries,E_F,-0.2139,-0.0000,-0.0007,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0007
7,Agriculture Total Production,A,-0.0057,0.0000,-0.0008,-0.0012,-0.0000,-0.0012,0.0000,0.0000,-0.0008
8,Agriculture Prod. For Biofuels,A_B,0.6161,0.0002,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,Agriculture Prod. For Food,A_F,-0.0303,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


## Land-use

In [52]:
df_base_policy.loc[:, 'Land-use effect'] = 0
df_base_policy.loc[2, 'Land-use effect'] = df_base_policy.loc[2, 'outcome']

print('Total effect:', round(df_base_policy['Land-use effect'].sum(), 5), '%')
print('Total effect (mHa):', round(3507*df_base_policy['Land-use effect'].sum()/100, 4))

df_base_policy

Total effect: -0.01104 %
Total effect (mHa): -0.3872


,index,variable,outcome,Aerosol effect,CO2 effect,Biodiv. effect,Biodiv. climate effect,Biodiv. incl. climate effect,Biogeochem. effect,Freshwater effect,Ocean acid. effect,Land-use effect
0,Land-Share Agriculture,L_A,0.0093,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,Land-Share Timber,L_T,0.0023,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,Land-Share Other,L_U,-0.0110,0.0000,0.0013,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0013,-0.0110
3,Fossil-Fuel Extracted,E,-0.3573,0.0000,-0.0322,-0.0008,-0.0000,-0.0008,0.0000,0.0000,-0.0322,0.0000
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610,-0.0135,-0.2122,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.2122,0.0000
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393,-0.0001,-0.0018,-0.0000,-0.0000,0.0000,-0.1393,0.0000,-0.0018,0.0000
6,Fossil-Fuel Use Fisheries,E_F,-0.2139,-0.0000,-0.0007,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0007,0.0000
7,Agriculture Total Production,A,-0.0057,0.0000,-0.0008,-0.0012,-0.0000,-0.0012,0.0000,0.0000,-0.0008,0.0000
8,Agriculture Prod. For Biofuels,A_B,0.6161,0.0002,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,Agriculture Prod. For Food,A_F,-0.0303,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


## Stratospheric Ozone
Anthropogenic ozone-depleting substances are essentially under control via the montreal protocol. Other than that
ozone depletion is positively correlated with climate change and NO2. The two later are largely due to agriculture and fossil fuel use and industry.

We do not consider cross-boundary effects and thus disregard from climate change.

Ozone is impacted by fossil fuel burning (energy services and fisheries), agricultural production (released e.g. from soil), burning of biofuels and industry.

We don´t quantify this process but rather assume a qualitative direction based on the above. If all affected variables become negative then the overall qualitative prediction will be negative.

In [53]:
df_base_policy.loc[:, 'Ozone effect'] = 0
# df_base_policy.loc[:, 'Ozone effect'] = -1*(df_base_policy.loc[:, 'CO2 effect'] < 0).apply(int)
#df_base_policy.loc[3, 'Ozone effect'] = min(max(-1*int(df_base_policy.loc[3, 'outcome'] < 0)+df_base_policy.loc[3, 'Ozone effect'], -1), 1)    # N02 fossil fuels
#df_base_policy.loc[7, 'Ozone effect'] = min(max(-1*int(df_base_policy.loc[7, 'outcome'] < 0)+df_base_policy.loc[7, 'Ozone effect'], -1), 1)    #  NO2 from agric.
#df_base_policy.loc[8, 'Ozone effect'] = min(max(-1*int(df_base_policy.loc[8, 'outcome'] < 0)+df_base_policy.loc[8, 'Ozone effect'], -1), 1)    #  NO2 from biofuel burning.
#df_base_policy.loc[19, 'Ozone effect'] = min(max(-1*int(df_base_policy.loc[19, 'outcome'] < 0)+df_base_policy.loc[19, 'Ozone effect'], -1), 1) # NO2 from industry.
df_base_policy.loc[3, 'Ozone effect'] = -1 if df_base_policy.loc[3, 'outcome'] < 0  else 1  # N02 fossil fuels
df_base_policy.loc[7, 'Ozone effect'] = -1 if df_base_policy.loc[7, 'outcome'] < 0  else 1    #  NO2 from agric.
df_base_policy.loc[8, 'Ozone effect'] = -1 if df_base_policy.loc[8, 'outcome'] < 0  else 1    #  NO2 from biofuel burning.
df_base_policy.loc[19, 'Ozone effect'] = -1 if df_base_policy.loc[19, 'outcome'] < 0  else 1 # NO2 from industry.

df_base_policy

,index,variable,outcome,Aerosol effect,CO2 effect,Biodiv. effect,Biodiv. climate effect,Biodiv. incl. climate effect,Biogeochem. effect,Freshwater effect,Ocean acid. effect,Land-use effect,Ozone effect
0,Land-Share Agriculture,L_A,0.0093,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0
1,Land-Share Timber,L_T,0.0023,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0
2,Land-Share Other,L_U,-0.0110,0.0000,0.0013,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0013,-0.0110,0
3,Fossil-Fuel Extracted,E,-0.3573,0.0000,-0.0322,-0.0008,-0.0000,-0.0008,0.0000,0.0000,-0.0322,0.0000,-1
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3610,-0.0135,-0.2122,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.2122,0.0000,0
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1393,-0.0001,-0.0018,-0.0000,-0.0000,0.0000,-0.1393,0.0000,-0.0018,0.0000,0
6,Fossil-Fuel Use Fisheries,E_F,-0.2139,-0.0000,-0.0007,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0007,0.0000,0
7,Agriculture Total Production,A,-0.0057,0.0000,-0.0008,-0.0012,-0.0000,-0.0012,0.0000,0.0000,-0.0008,0.0000,-1
8,Agriculture Prod. For Biofuels,A_B,0.6161,0.0002,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1
9,Agriculture Prod. For Food,A_F,-0.0303,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0


# Chemical pollution

Inline with Stratospheric ozone we do a qualitative assesment on this process based on model variables fossil fuel extracted, agriculture total and final goods.


In [270]:
df_base_policy.loc[:, 'Chem. effect'] = 0
df_base_policy.loc[3, 'Chem. effect'] = -1*int(df_base_policy.loc[3, 'outcome'] < 0)+df_base_policy.loc[3, 'Chem. effect']   # fossil fuels
df_base_policy.loc[7, 'Chem. effect'] = -1*int(df_base_policy.loc[7, 'outcome'] < 0)+df_base_policy.loc[7, 'Chem. effect']   # agric.
df_base_policy.loc[19, 'Chem. effect'] = -1*int(df_base_policy.loc[19, 'outcome'] < 0)+df_base_policy.loc[19, 'Chem. effect']   # industry.


df_base_policy

,index,variable,outcome,Aerosol effect,CO2 effect,Biodiv. effect,Biodiv. climate effect,Biodiv. incl. climate effect,Biogeochem. effect,Freshwater effect,Ocean acid. effect,Land-use effect,Ozone effect,Chem. effect
0,Land-Share Agriculture,L_A,-0.0374,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0
1,Land-Share Timber,L_T,0.0207,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0
2,Land-Share Other,L_U,0.0431,-0.0001,-0.0053,0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0053,0.0431,0,0
3,Fossil-Fuel Extracted,E,-0.3590,0.0000,-0.0324,-0.0008,-0.0000,-0.0008,0.0000,0.0000,-0.0324,0.0000,-1,-1
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.3620,-0.0135,-0.2128,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.2128,0.0000,0,0
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.1841,-0.0001,-0.0024,-0.0000,-0.0000,0.0000,-0.1841,0.0000,-0.0024,0.0000,0,0
6,Fossil-Fuel Use Fisheries,E_F,-0.2230,-0.0000,-0.0007,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0007,0.0000,0,0
7,Agriculture Total Production,A,-0.0449,0.0000,-0.0062,-0.0092,-0.0000,-0.0092,0.0000,0.0000,-0.0062,0.0000,-1,-1
8,Agriculture Prod. For Biofuels,A_B,-1.0078,-0.0003,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-1,0
9,Agriculture Prod. For Food,A_F,-0.0068,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0


## Final output

In [271]:
df_final_impact = df_base_policy[['index', 'variable', 'CO2 effect', 'Biodiv. incl. climate effect', 'Biogeochem. effect', 'Freshwater effect', 
                                  'Ocean acid. effect', 'Land-use effect', 'Aerosol effect', 'Ozone effect', 'Chem. effect']]
df_final_impact.to_excel('PB_impacts.xlsx')
df_final_impact

,index,variable,CO2 effect,Biodiv. incl. climate effect,Biogeochem. effect,Freshwater effect,Ocean acid. effect,Land-use effect,Aerosol effect,Ozone effect,Chem. effect
0,Land-Share Agriculture,L_A,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0
1,Land-Share Timber,L_T,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0
2,Land-Share Other,L_U,-0.0053,0.0000,0.0000,0.0000,-0.0053,0.0431,-0.0001,0,0
3,Fossil-Fuel Extracted,E,-0.0324,-0.0008,0.0000,0.0000,-0.0324,0.0000,0.0000,-1,-1
4,Fossil-Fuel Use Energy Serv.,E_{\mathcal{E}},-0.2128,0.0000,0.0000,0.0000,-0.2128,0.0000,-0.0135,0,0
5,Fossil-Fuel Use Fertilizer Prod.,E_P,-0.0024,0.0000,-0.1841,0.0000,-0.0024,0.0000,-0.0001,0,0
6,Fossil-Fuel Use Fisheries,E_F,-0.0007,0.0000,0.0000,0.0000,-0.0007,0.0000,-0.0000,0,0
7,Agriculture Total Production,A,-0.0062,-0.0092,0.0000,0.0000,-0.0062,0.0000,0.0000,-1,-1
8,Agriculture Prod. For Biofuels,A_B,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0003,-1,0
9,Agriculture Prod. For Food,A_F,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0


### Generate pb impact tables

In [272]:
import importlib
from web_model import pb_table_generators as pt
importlib.reload(pt)
pt.model_pbimpact_result_table(df_final_impact)

\begin{tabular}{llllllllll} 
  & \textit{Climate} & \textit{Biodiv.} & \textit{Biochem.} & \textit{Freshw.} & \textit{Ocean acid.} & \textit{Land-use}  & \textit{Aerosols} & \textit{Ozone} & \textit{Chem.} \\ 
\hline
Land-Share Agriculture ($L_A$)  & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0 & 0 \\ 
Land-Share Timber ($L_T$)  & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0 & 0 \\ 
Land-Share Other ($L_U$)  & -0.0053 & 0.0 & 0.0 & 0.0 & -0.0053 & 0.0431 & -0.0001 & 0 & 0 \\ 
Fossil-Fuel Extracted ($E$)  & -0.0324 & -0.0008 & 0.0 & 0.0 & -0.0324 & 0.0 & 0.0 & -1 & -1 \\ 
Fossil-Fuel Use Energy Serv. ($E_{\mathcal{E}}$)  & -0.2128 & 0.0 & 0.0 & 0.0 & -0.2128 & 0.0 & -0.0135 & 0 & 0 \\ 
Fossil-Fuel Use Fertilizer Prod. ($E_P$)  & -0.0024 & 0.0 & -0.1841 & 0.0 & -0.0024 & 0.0 & -0.0001 & 0 & 0 \\ 
Fossil-Fuel Use Fisheries ($E_F$)  & -0.0007 & 0.0 & 0.0 & 0.0 & -0.0007 & 0.0 & -0.0 & 0 & 0 \\ 
Agriculture Total Production ($A$)  & -0.0062 & -0.0092 & 0.0 & 0.0 & -0.0062 & 0.0 & 0.0 & -1 & -

### Generate parameter tables

In [274]:
import importlib
from web_model import pb_table_generators as pt
importlib.reload(pt)
#pt.model_pbimpact_result_table(df_final_impact)

<module 'web_model.pb_table_generators' from '/Users/gustavengstrom/Google Drive Work/jobb/Ragnar Soderberg/model/web_model/pb_table_generators.py'>

In [275]:
pt.param_expshare_table()
pt.param_qshare_table()
pt.param_elast_table()

25
\begin{table}[!ht]
\centering
\caption{\textbf{Parameters: expenditure shares (source: GTAP)}}
\begin{tabular}{ll} 
\hline
\textit{Expenditure share} & \textit{value} \\
\hline
$\prodel{A}{L_A}$ & 19.2\%  \\
$\prodel{\tilde{L}_A}{P}$ & 8.0\%   \\ 
$\prodel{\tilde{L}_A}{W}$ & 2.4\%  \\ 
$\prodel{\tilde{L}_A}{\mathcal{E}_A}$ & 4.1\%  \\ 
$\prodel{\mathcal{E}}{A_B}$ & 0.4\%   \\
$\prodel{\mathcal{E}}{E_\mathcal{E}}$ & 94.3\%   \\
$\prodel{U}{\mathcal{F}}$ & 12.3\%   \\ 
$\prodel{\mathcal{F}}{F}$ & 3.4\%   \\ 
$\prodel{\tilde{\mathcal{F}}}{Y}$ & 99.1\%   \\ 
$\prodel{\tilde{\mathcal{F}}}{L_U}$ & 1.7\%   \\ 
$\prodel{T}{L_T}$ & 37.5\%   \\
$\prodel{Y}{\mathcal{E}_Y}$ & 6.4\%   \\
$\prodel{P}{E_P}$ & 10.9\%   \\
$\prodel{P}{\mathcal{P}}$ & 31.3\%   \\
$\prodel{F}{E_F}$ & 22.8\%   \\
\hline
\end{tabular}
\label{Tab:FactorShares}
\end{table}
\begin{table}[!ht]
\centering
\caption{\textbf{Parameters - Quantity shares}}
\begin{tabular}{lll} 
\hline
\textit{Parameter} & \textit{source} & \text

### Generate sensitivity table

In [299]:
import importlib
from web_model import pb_table_generators as pt
importlib.reload(pt)
pt.model_variable_sensitivity_table(var_desc_dict, df_carbontax_quantities, df_biofuel_quantities)

\begin{tabular}{lrrrr}
\hline
& \multicolumn{2}{c}{}  & \multicolumn{2}{c}{\textit{Carbon Tax +}}  \\ 
& \multicolumn{2}{c}{\textit{Carbon tax}}  & \multicolumn{2}{c}{\textit{Biofuel policy} }  \\ 
\textit{Variable} & \textit{Min} & \textit{Max} & \textit{Min} & \textit{Max}  \\ 
\hline
\multicolumn{5}{l}{\textit{Agricultural Sector: Production}} \\ 
\hspace{0.2cm}Total & -0.0172 & 0.0305 & -0.0814 & -0.0229 \\ 
\hspace{0.2cm}Biofuels & 0.1898 & 1.5558 & -1.6501 & -0.3551 \\ 
\hspace{0.2cm}Food & -0.0599 & -0.0157 & -0.0395 & 0.021 \\ 
\multicolumn{5}{l}{\textit{Agricultural Sector: Inputs}} \\ 
\hspace{0.2cm}Land-Share Agriculture & -0.0109 & 0.0574 & -0.0732 & -0.0001 \\ 
\hspace{0.2cm}Energy in Agriculture & -0.5853 & -0.1067 & -0.6477 & -0.1476 \\ 
\hspace{0.2cm}Fertilizer Production & -0.0426 & 0.0141 & -0.1048 & -0.0352 \\ 
\hspace{0.2cm}Water Production & -0.0115 & 0.041 & -0.073 & -0.0128 \\ 
\multicolumn{5}{l}{\textit{Energy-related sectors and services}} \\ 
\hspace{0.2cm}Fos

### most sensitivite parameter check

In [55]:
import copy
import pandas as pd
mse_param_result_set = []
modified_param_list = copy.deepcopy(param_list)
parameter_perturbation = 0.25
for idx, par in enumerate(param_list):
    value = par['values'][2] if isinstance(par['values'], list) else par['values']
    
    if  'sigma_' in par['keys'] or 'Lambda_' in par['keys'] or 'V_' in par['keys'] or 'tau_' in par['keys']:
        modified_param_list[idx]['values'] = value*(1-parameter_perturbation)
        df_result = model.solve(modified_param_list)
        mse = (abs(df_baseline_model_result["mean value"] - df_result["mean value"])).mean()
        wmse = abs((df_result['mean value']-df_baseline_model_result['mean value'])/df_baseline_model_result['mean value']).mean()
        mse_param_result_set.append({'param': par['keys'], 'mse': mse, 'wmse': wmse, 'end': 'low' })
        modified_param_list[idx]['values']  = value
   
        result_final = sum_pb_results(df_result)
        pb_unchanged = all((baseline_model_result_final>0) == (result_final>0))
        assert pb_unchanged
        
        assert all([modified_param_list[i]['values']==(p['values']) for i, p in enumerate(param_list) if isinstance(p, int)])
        assert all([modified_param_list[i]['values']==(p['values'][2]) for i, p in enumerate(param_list) if isinstance(p, list)])
        
        modified_param_list[idx]['values'] = value*(1+parameter_perturbation)
        df_result = model.solve(modified_param_list)
        mse = (abs(df_baseline_model_result["mean value"] - df_result["mean value"])).mean()
        wmse = abs((df_result['mean value']-df_baseline_model_result['mean value'])/df_baseline_model_result['mean value']).mean()
        mse_param_result_set.append({'param': par['keys'], 'mse': mse, 'wmse': wmse, 'end': 'high' })
        modified_param_list[idx]['values']  = value
        
        result_final = sum_pb_results(df_result)
        pb_unchanged = all((baseline_model_result_final>0) == (result_final>0))
        assert pb_unchanged
        
        result_final = sum_pb_results(df_result)
        pb_change = all((baseline_model_result_final>0) == (result_final>0))
        assert all([modified_param_list[i]['values']==p['values'] for i, p in enumerate(param_list) if isinstance(p, int)])
        assert all([modified_param_list[i]['values']==(p['values'][2]) for i, p in enumerate(param_list) if isinstance(p, list)])
        
        #print(par['keys'], par['values'])
        


mse_param_result_set.sort(key=lambda x: x['wmse'], reverse=True)
pd.DataFrame(mse_param_result_set)

NameError: name 'param_list' is not defined